In [174]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import numpy as np

In [13]:
URL = "https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=12"
HEADERS = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36" }

In [14]:
def get_HTML(URL, params = None):
    r = requests.get(URL, headers = HEADERS, params = params)
    return r

In [231]:
def parse(URL):
    stock_params_dict = {}
    html = get_HTML(URL)
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, "html.parser")
        items = soup.find_all("a", class_ = "Link-module__link__Wv1b Link-module__link_theme_default_gf5wq")
        
        stock_title_list = pd.Series([item.find('div', class_ = "Caption__caption_cyYZT") \
                         for item in items if item.find('div', class_ = "Caption__caption_cyYZT") ]).apply(lambda i: i.text)
        stock_ticker_list = pd.Series([item.find('div', class_ = "Caption__subcaption_xTAKS") \
                         for item in items if item.find('div', class_ = "Caption__subcaption_xTAKS") ]).apply(lambda i: i.text)
        stock_price_list = [price.replace("\xa0","") for price in pd.Series([item.find('div', class_ = "SecurityColumn__cellPriceSecurities_cswKs") \
                         for item in items if item.find('div', class_ = "SecurityColumn__cellPriceSecurities_cswKs") ]).apply(lambda i: i.text)]
        stock_yield_listok = pd.Series([item.find('span', class_ = "Money-module__money_UwC2N") \
                         for item in items if item.find('span', class_ = "Money-module__money_UwC2N") ]).apply(lambda i: i.text)
        stock_yield_list = [yld.replace("\xa0","") for yld in stock_yield_listok[::2]]
        for i in range(12):
            stock_params_dict[stock_title_list[i]] = [stock_ticker_list[i],stock_price_list[i], stock_yield_list[i] ]
    else:
        print("Error")
    return  stock_params_dict


In [232]:
URL_all = "https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=12"

In [233]:
stock_params_dict  = parse(URL_all)

In [18]:
all_URL = []
for i in range(1,6):
    all_URL.append("https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=" + str(12*i))

In [234]:
df = np.transpose(pd.DataFrame(stock_params_dict))  
df.columns = ["Ticker","Price","Yield"]
df["Price"] = df["Price"].map(lambda i: float(i[:-1].replace(",",".")))

df

,Ticker,Price,Yield
1-800-Flowers.com Inc,FLWS,34.36,"+5,04$"
10X Genomics Inc,TXG,160.27,"−8,24$"
1Life Healthcare,ONEM,21.86,0$
2seventy bio,TSVT,35.30,0$
2U Inc,TWOU,25.87,"−10,03$"
3D Systems Corp,DDD,25.62,0$
3M Company,MMM,182.90,"−8,1$"
51job,JOBS,51.13,"−12,36$"
8x8 Inc,EGHT,21.98,"−10,02$"
A. O. Smith Corporation,AOS,82.75,"+15,29$"


18